In [1]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 

# ZILINGO Scraping

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

## Top 400 per category

In [142]:
# Scraping the page
def scrape_detail_page(url,category,collection):
    
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    db = conn.zilingo
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    result = pd.DataFrame()
    time.sleep(5)

    # First scroll to the end of the page
    # Click view more
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    error = 0
    scroll = 0
    start = 0
    while (error<10 and scroll<11):
        print("scroll: " + str(scroll))
        if(error == 0):
            # Looping item
            item = driver.find_elements_by_css_selector('.m-product-card.m-product-card--focus')
            start = (scroll*48)
            end = (scroll+1)*48
            scroll = scroll+1
            if(len(item)<end):
                end = len(item)
            for x in range(start,end):
                try:
                    discounted_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__discount')[0].text))
                    origin_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__old')[0].text))
                except:
                    origin_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__original')[0].text))
                    discounted_price = origin_price
                name = item[x].find_elements_by_css_selector('.info__name')[0].text
                image = item[x].find_elements_by_css_selector('img')[0].get_attribute('src')

                while(image=='//:0'):
                    element=item[x]
                    element.location_once_scrolled_into_view
                    time.sleep(2)
                    image = item[x].find_elements_by_css_selector('img')[0].get_attribute('src')
                    
                sku = re.findall("details/(.*?)&",item[x].find_elements_by_css_selector('.m-product-card__link')[0].get_attribute('href'))[0]
                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date}])
                result = result.append(new_result)

                # save to mongodb
                collection.insert_many(new_result.to_dict('records'))

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight <= lastHeight:
            error = error+1
            print("error: " + str(error))
            time.sleep(5)
        else:
            error = 0
        lastHeight = newHeight


    driver.close()
    return result

In [143]:
# MAIN
zilingo_category = pd.read_csv('C:\\Users\\Annisa\\Stream\\Product Development - Analysis - QBR Jul19\\Data\\scraping\\zilingo_category.csv',sep=";")
result = pd.DataFrame()
for x in range(44,len(zilingo_category)):
    category = zilingo_category.iloc[x]
    print(category['category'])
    url = category['url']+"?sort_order=popular"
    result = scrape_detail_page(url,category['category'],category['collection'])
    print(result.head())

swimwear
Connected successfully!!!
scroll: 0
scroll: 1
scroll: 2
scroll: 3
scroll: 4
scroll: 5
scroll: 6
scroll: 7
error: 1
scroll: 8
error: 2
scroll: 8
error: 3
scroll: 8
error: 4
scroll: 8
error: 5
scroll: 8
error: 6
scroll: 8
error: 7
scroll: 8
error: 8
scroll: 8
error: 9
scroll: 8
error: 10
   category discounted_price  \
0  swimwear           354000   
0  swimwear           238000   
0  swimwear           354000   
0  swimwear           354000   
0  swimwear           400000   

                                               image  \
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   

                                                name origin_price  \
0                   3pcs Set Mermaid Tail Swimsuit B       506000   
0  Baju Renang anak cewek - Mermaid 

scroll: 0
scroll: 1
scroll: 2
scroll: 3
scroll: 4
scroll: 5
scroll: 6
scroll: 7
scroll: 8
scroll: 9
scroll: 10
  category discounted_price  \
0  jackets           138000   
0  jackets           449000   
0  jackets           108000   
0  jackets           101000   
0  jackets           164000   

                                               image  \
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   

                                                name origin_price  \
0                      Jaket Anak Denim Keren Mywear       250000   
0                    Children's cotton jacket hooded       642000   
0           Mon Cheri Cardigan classic -Toddler Size       269000   
0  Bella Fashion, Baju anak Tompege 1811, Jaket a...       104000   
0  INFIKIDS Jaket An

## By Brand

In [3]:
# Scraping the page
def scrape_detail_page2(url,category,collection,brand):
    
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    db = conn.zilingo
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    result = pd.DataFrame()
    time.sleep(5)

    # First scroll to the end of the page
    # Click view more
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    error = 0
    scroll = 0
    start = 0
    while (error<10):
        print("scroll: " + str(scroll))
        if(error == 0):
            # Looping item
            item = driver.find_elements_by_css_selector('.m-product-card.m-product-card--focus')
            start = (scroll*48)
            end = (scroll+1)*48
            scroll = scroll+1
            if(len(item)<end):
                end = len(item)
            for x in range(start,end):
                try:
                    discounted_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__discount')[0].text))
                    origin_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__old')[0].text))
                except:
                    origin_price = re.sub("\.","",re.sub("RP ","",item[x].find_elements_by_css_selector('.amount__original')[0].text))
                    discounted_price = origin_price
                name = item[x].find_elements_by_css_selector('.info__name')[0].text
                image = item[x].find_elements_by_css_selector('img')[0].get_attribute('src')

                while(image=='//:0'):
                    element=item[x]
                    element.location_once_scrolled_into_view
                    time.sleep(2)
                    image = item[x].find_elements_by_css_selector('img')[0].get_attribute('src')
                    
                sku = re.findall("details/(.*?)&",item[x].find_elements_by_css_selector('.m-product-card__link')[0].get_attribute('href'))[0]
                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date,'brand':brand}])
                result = result.append(new_result)

                # save to mongodb
                collection.insert_many(new_result.to_dict('records'))

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight <= lastHeight:
            error = error+1
            print("error: " + str(error))
            time.sleep(5)
        else:
            error = 0
        lastHeight = newHeight


    driver.close()
    return result

In [8]:
# MAIN
#zilingo_category = pd.read_csv('C:\\Users\\Annisa\\Stream\\Product Development - Analysis - QBR Jul19\\Data\\scraping\\zilingo_category.csv',sep=";")
zilingo_category = pd.read_csv('C:/Users/tamu/SharePoint/Product Development - Documents/Merchandising Analytics/Analysis - QBR Jul19/Data/scraping/zilingo_category.csv',sep=';')
result = pd.DataFrame()
for x in range(0,len(zilingo_category)):
    category = zilingo_category.iloc[x]
    print(category['category'])
    brand = "MANGO"
    url = category['url']+"?brand=BRD7534894"
    collection = category['collection']+"_by_brand"
    result = scrape_detail_page2(url,category['category'],collection,brand)
    print(result.head())

tops_and_tunics
Connected successfully!!!
scroll: 0
scroll: 1
error: 1
scroll: 2
error: 2
scroll: 2
error: 3
scroll: 2
error: 4
scroll: 2
error: 5
scroll: 2
error: 6
scroll: 2
error: 7
scroll: 2
error: 8
scroll: 2
error: 9
scroll: 2
error: 10
   brand         category discounted_price  \
0  MANGO  tops_and_tunics           399000   
0  MANGO  tops_and_tunics           599000   
0  MANGO  tops_and_tunics           599000   
0  MANGO  tops_and_tunics           479000   
0  MANGO  tops_and_tunics           139000   

                                               image  \
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   

                          name origin_price scraping_date  \
0         MANGO Chiffon blouse       399000    2019-05-28   
0     MANGO Buttons c

scroll: 2
error: 2
scroll: 2
error: 3
scroll: 2
error: 4
scroll: 2
error: 5
scroll: 2
error: 6
scroll: 2
error: 7
scroll: 2
error: 8
scroll: 2
error: 9
scroll: 2
error: 10
   brand category discounted_price  \
0  MANGO  jackets           299000   
0  MANGO  jackets           599000   
0  MANGO  jackets           599000   
0  MANGO  jackets           699000   
0  MANGO  jackets           299000   

                                               image  \
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   

                        name origin_price scraping_date  \
0  MANGO Ribbed knit sweater       299000    2019-05-28   
0  MANGO Medium denim jacket       599000    2019-05-28   
0  MANGO Medium denim jacket       599000    2019-05-28   
0  MANGO Flecked rib sweat

scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1
error: 7
scroll: 1
error: 8
scroll: 1
error: 9
scroll: 1
error: 10
   brand category discounted_price  \
0  MANGO    boots           999000   

                                               image  \
0  https://dj7u9rvtp3yka.cloudfront.net/products/...   

                            name origin_price scraping_date  \
0  MANGO Decorative buckle boots       999000    2019-05-28   

                         sku  
0  PRO2203253465?color=black  
sports-shoes
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1
error: 7
scroll: 1
error: 8
scroll: 1
error: 9
scroll: 1
error: 10
Empty DataFrame
Columns: []
Index: []
casual-shoes
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1

error: 9
scroll: 1
error: 10
Empty DataFrame
Columns: []
Index: []
footwear
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1
error: 7
scroll: 1
error: 8
scroll: 1
error: 9
scroll: 1
error: 10
Empty DataFrame
Columns: []
Index: []
nightsuits
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1
error: 7
scroll: 1
error: 8
scroll: 1
error: 9
scroll: 1
error: 10
Empty DataFrame
Columns: []
Index: []
swimwear
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 1
error: 6
scroll: 1
error: 7
scroll: 1
error: 8
scroll: 1
error: 9
scroll: 1
error: 10
Empty DataFrame
Columns: []
Index: []
innerwear
Connected successfully!!!
scroll: 0
error: 1
scroll: 1
error: 2
scroll: 1
error: 3
scroll: 1
error: 4
scroll: 1
error: 5
scroll: 